In [67]:
import torch
import torch.nn as nn
import sys
sys.path.append('../')
import pandas as pd


from utils.GetData import Data2
import numpy as np

In [68]:
PATH = '../in/train.csv'
df = pd.read_csv(PATH)

In [69]:
data = Data2()

In [70]:
dfx = data.get_data2(df.copy())

In [71]:
feature = dfx.drop('SalePrice', axis=1)
y = dfx['SalePrice']

In [72]:
class Model(nn.Module):

    def __init__(self, D_in, D_out):
        super(Model, self).__init__()

        self.c1 = nn.Linear(D_in, 256)
        self.c2 = nn.ReLU()
        self.c3 = nn.Dropout(p=0.1)
        self.c4 = nn.Linear(256, 256)
        self.c5 = nn.ReLU()
        self.c6 = nn.Dropout(p=0.1)
        self.c7 = nn.Linear(256, 128)
        self.c8 = nn.ReLU()
        self.c9 = nn.Dropout(p=0.1)
        self.c10 = nn.Linear(128, 128)
        self.c11 = nn.ReLU()
        self.c12 = nn.Dropout(p=0.1)
        self.c13 = nn.Linear(128, D_out)

    def forward(self, X):
        X = self.c1(X)
        X = self.c2(X)
        X = self.c3(X)
        X = self.c4(X)
        X = self.c5(X)
        X = self.c6(X)
        X = self.c7(X)
        X = self.c8(X)
        X = self.c9(X)
        X = self.c10(X)
        X = self.c11(X)
        X = self.c12(X)
        X = self.c13(X)
        return X
        

In [73]:
feature.shape[1]

207

In [74]:
model = Model(feature.shape[1], 1)

In [75]:
X = torch.from_numpy(feature.astype('float64').values)
y = torch.from_numpy(y.astype('float64').values)

In [76]:
X = X.type(torch.FloatTensor)
y = y.type(torch.FloatTensor)

In [77]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

In [88]:
epochs = 500
batch_size = 100
log_each = 1
l = []
model.train()
batches = len(X) // batch_size
for e in range(1, epochs + 1):
    _l = []
    for b in range(batches):
        x_b = X[b*batch_size:(b+1)*batch_size]
        y_b = y[b*batch_size:(b+1)*batch_size]

        y_pred = model(x_b)
        loss = criterion(y_pred, y_b)

        _l.append(loss.item())

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

    l.append(np.mean(_l))
    if not e % log_each:
        print(f'Epoch {e}/{epochs} Loss {np.mean(l):.5f}')


c:\Users\daniel\Desktop\practica\Advanced_Regression_Techniques\renv\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/500 Loss 6590039314.28571
Epoch 2/500 Loss 6605700800.00000
Epoch 3/500 Loss 6610329252.57143
Epoch 4/500 Loss 6611846715.42857
Epoch 5/500 Loss 6609676189.25714
Epoch 6/500 Loss 6606728225.52381
Epoch 7/500 Loss 6604549958.53061
Epoch 8/500 Loss 6601942918.85714
Epoch 9/500 Loss 6601954423.87302
Epoch 10/500 Loss 6600726581.02857
Epoch 11/500 Loss 6599609688.10390
Epoch 12/500 Loss 6601290823.61905
Epoch 13/500 Loss 6603022106.72528
Epoch 14/500 Loss 6604325304.16327
Epoch 15/500 Loss 6606508708.57143
Epoch 16/500 Loss 6606830986.28571
Epoch 17/500 Loss 6607999749.37815
Epoch 18/500 Loss 6608732616.12698
Epoch 19/500 Loss 6608836046.91729
Epoch 20/500 Loss 6608709104.45714
Epoch 21/500 Loss 6608302066.93877
Epoch 22/500 Loss 6608267303.06493
Epoch 23/500 Loss 6609663151.70186
Epoch 24/500 Loss 6610696965.33333
Epoch 25/500 Loss 6610546548.29714
Epoch 26/500 Loss 6611437485.71428
Epoch 27/500 Loss 6612058898.28571
Epoch 28/500 Loss 6612972912.97959
Epoch 29/500 Loss 6612706714.

In [89]:
pred_example = pd.read_csv('../in/sample_submission.csv')

In [90]:
test = pd.read_csv('../in/test.csv')

In [92]:
test_df = data.get_data2(test.copy(), fit=False)

In [82]:
test_df.shape

(1459, 207)

In [93]:
X_test = torch.from_numpy(test_df.astype('float64').values)
X_test = X_test.type(torch.FloatTensor)

In [94]:
model.eval()

Model(
  (c1): Linear(in_features=207, out_features=256, bias=True)
  (c2): ReLU()
  (c3): Dropout(p=0.1, inplace=False)
  (c4): Linear(in_features=256, out_features=256, bias=True)
  (c5): ReLU()
  (c6): Dropout(p=0.1, inplace=False)
  (c7): Linear(in_features=256, out_features=128, bias=True)
  (c8): ReLU()
  (c9): Dropout(p=0.1, inplace=False)
  (c10): Linear(in_features=128, out_features=128, bias=True)
  (c11): ReLU()
  (c12): Dropout(p=0.1, inplace=False)
  (c13): Linear(in_features=128, out_features=1, bias=True)
)

In [95]:
y_predict = model(X_test)

In [96]:
y_predict = y_predict.detach().numpy()

In [97]:
predict = pd.DataFrame(y_predict, columns=['SalePrice'])
predict['Id'] = index=pred_example.Id
predict = predict.set_index('Id')
predict.to_csv('../out/MLP_500.csv')
predict.head()

,SalePrice
Id,
1461,174267.859375
1462,167103.250000
1463,177965.890625
1464,177942.000000
1465,175270.484375
